In [1]:
#Import required libraries
import gspread
import sys
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import numpy as np
import urllib
import sqlalchemy
from gspread_dataframe import set_with_dataframe
from gspread_dataframe import get_as_dataframe

In [2]:
#Import project specific functions
from column_map import column_map
from yesno_functions import *
from format_datetime import *

In [3]:
#Import shared functions
sys.path.append('../..')
from IPM_Shared_Code_public.Python.google_creds_functions import create_assertion_session
from IPM_Shared_Code_public.Python.utils import get_config
from IPM_Shared_Code_public.Python.delta_functions import *
from IPM_Shared_Code_public.Python.sql_functions import *

C:\ProgramData\Anaconda3\lib\site-packages\authlib\client\__init__.py:12: AuthlibDeprecationWarning: Deprecate "authlib.client", USE "authlib.integrations.requests_client" instead.
It will be compatible before version 1.0.
Read more <https://git.io/Jeclj#file-rn-md>
  deprecate('Deprecate "authlib.client", USE "authlib.integrations.requests_client" instead.', '1.0', 'Jeclj', 'rn')


### Use the config file to setup connections

In [4]:
config = get_config('c:\Projects\config.ini')

driver = config['srv']['driver']
server = config['srv']['server']
dwh = config['db']['crowdsdb']
cred_file = config['google']['path_to_file']

In [5]:
con_string = 'Driver={' + driver + '};Server=' + server +';Database=' + dwh + ';Trusted_Connection=Yes;'
params = urllib.parse.quote_plus(con_string)
engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

### Execute the function to get the renamed columns for this sheet

In [6]:
#Call the column map function to get the dictionary to be used for renaming and subsetting the columns
col_rename = column_map('patrol_dpr')

In [7]:
#Because of duplicate column names these columns are renamed based on the column index and the keys and 
#values need to be switched
col_rename = {v[0]: k for k, v in col_rename.items()}

In [8]:
cols = list(col_rename.values())

### Read the current data from SQL

In [9]:
sql = 'select * from crowdsdb.dbo.tbl_dpr_patrol'

In [24]:
patrol_sql = (pd.read_sql(con = engine, sql = sql)
              .drop(columns = ['patrol_id', 'patroncount'])
              .fillna(value = np.nan, axis = 1))

In [25]:
patrol_sql['encounter_timestamp'] = format_datetime(patrol_sql['encounter_timestamp'])
patrol_sql['encounter_datetime'] = format_datetime(patrol_sql['encounter_datetime'])

In [11]:
sql_cols = list(patrol_sql.columns.values)

In [26]:
patrol_sql.head()

,encounter_timestamp,encounter_datetime,site_id,location_adddesc,park_division,visit_reason,firstname_1,lastname_1,firstname_2,lastname_2,...,sd_patronscomplied,sd_patronsnocomply,sd_amenity,summonscount_a01,summonscount_a03,summonscount_a04,summonscount_a22,other_summonstype,other_summonscount,borough
0,2020-05-29 17:25:27,2020-05-29 17:08:00,X092-09,NaN,Rangers,Standard Patrol,Brittany,Nulph,Ilana,Weinstein,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bronx
1,2020-05-29 17:24:51,2020-05-29 17:23:00,X073,NaN,Rangers,Standard Patrol,Hunter,Mathis,Albert,Seto,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bronx
2,2020-05-29 17:24:50,2020-05-29 16:49:00,X092-06,NaN,Rangers,Standard Patrol,Brittany,Nulph,Ilana,Weinstein,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bronx
3,2020-05-29 17:24:01,2020-05-29 16:40:00,X092-26,NaN,Rangers,Standard Patrol,Brittany,Nulph,Ilana,Weinstein,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bronx
4,2020-05-29 17:23:27,2020-05-29 17:10:00,M376,NaN,PEP,Fixed Post,Anna,Williams,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Manhattan


In [27]:
hash_rows(patrol_sql, exclude_cols = ['encounter_timestamp'], hash_name = 'row_hash')

### Read the site reference list from SQL

In [28]:
sql = 'select * from crowdsdb.dbo.tbl_ref_sites'

In [29]:
site_ref = pd.read_sql(con = engine, sql = sql)[['site_id', 'site_desc', 'borough']]

### Read the latest data from Google Sheets

In [30]:
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name(cred_file, scope)
client = gspread.authorize(creds)

In [31]:
sheet = client.open('COMBINED Patrol Reporting Responses')

In [32]:
ws = sheet.worksheet('MASTER')

In [33]:
#Read the worksheet as a data frame, rename the columns and subset the columns to only include those
#in the column list
patrol = (get_as_dataframe(ws, evaluate_formulas = True, header= None)
          #Always remove row 0 with the column headers
          .iloc[1:]
          .rename(columns = col_rename)
          .fillna(value = np.nan, axis = 1))[cols]

In [ ]:
patrol['encounter_timestamp'] = format_datetime(patrol['encounter_timestamp'])
patrol['encounter_datetime'] = format_datetime(patrol['encounter_datetime'])

In [34]:
patrol.head()

,encounter_timestamp,encounter_datetime,site_desc,location_adddesc,park_division,firstname_1,lastname_1,firstname_2,lastname_2,firstname_3,...,sd_pdassist,sd_pdcontact,sd_comments,summonscount_a01,summonscount_a03,summonscount_a04,summonscount_a22,other_summonstype,other_summonscount,borough
1,6/1/2020 11:29:57,6/1/2020 11:12:00,Great Kills Park,cedar grove beach,Rangers,Emily,Nacchio,Glen,Higgins,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Staten Island
2,6/1/2020 11:28:49,6/1/2020 11:27:00,Tottenville Pool,playground,Rangers,anthony,denicola,Nicholas,wyant,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Staten Island
3,6/1/2020 11:26:41,6/1/2020 11:30:00,Captain Tilly Park-Captain Tilly Playground,NaN,PEP,SGT IRIS,Hernandez,Richard,Diaz,none,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Queens
4,6/1/2020 11:26:28,6/1/2020 11:17:00,Tremont Park,NaN,PEP,Tezra,Swinea,Oscar,Marrero,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bronx
5,6/1/2020 11:24:54,6/1/2020 10:56:00,Veterans Park,NaN,PEP,J,Krueger,S,Villegas,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Staten Island


In [35]:
yesno = ['closed_education', 'closed_outcome', 'closed_summonsissued', 'closed_pdassist',
         'closed_pdcontact', 'sd_summonsissued', 'sd_pdassist', 'sd_pdcontact']

In [36]:
yesno_cols(patrol, yesno)

In [37]:
#Remove any rows with no data, presumably these are rows with no timestamp
patrol = patrol[patrol['encounter_timestamp'].notna()]

In [38]:
patrol = patrol.merge(site_ref, how = 'left', on = ['site_desc', 'borough'])[sql_cols]

In [39]:
hash_rows(patrol, exclude_cols = ['site_id', 'encounter_timestamp'], hash_name = 'row_hash')

In [40]:
patrol_deltas = (check_deltas(new_df = patrol, old_df = patrol_sql, on = ['site_id', 'encounter_timestamp'], 
                              hash_name = 'row_hash', dml_col = 'dml_verb'))[sql_cols + ['dml_verb']]

In [41]:
patrol_inserts = patrol_deltas[patrol_deltas['dml_verb'] == 'I'][sql_cols]

In [42]:
patrol_inserts.head()

,encounter_timestamp,encounter_datetime,site_id,location_adddesc,park_division,visit_reason,firstname_1,lastname_1,firstname_2,lastname_2,...,sd_patronscomplied,sd_patronsnocomply,sd_amenity,summonscount_a01,summonscount_a03,summonscount_a04,summonscount_a22,other_summonstype,other_summonscount,borough
0,6/1/2020 11:29:57,6/1/2020 11:12:00,R016,cedar grove beach,Rangers,Standard Patrol,Emily,Nacchio,Glen,Higgins,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Staten Island
1,6/1/2020 11:28:49,6/1/2020 11:27:00,R110,playground,Rangers,Standard Patrol,anthony,denicola,Nicholas,wyant,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Staten Island
2,6/1/2020 11:26:41,6/1/2020 11:30:00,NaN,NaN,PEP,Standard Patrol,SGT IRIS,Hernandez,Richard,Diaz,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Queens
3,6/1/2020 11:26:28,6/1/2020 11:17:00,X010A,NaN,PEP,Standard Patrol,Tezra,Swinea,Oscar,Marrero,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bronx
4,6/1/2020 11:24:54,6/1/2020 10:56:00,R019,NaN,PEP,Standard Patrol,J,Krueger,S,Villegas,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Staten Island


In [48]:
patrol_inserts.to_sql('tbl_dpr_patrol', engine, index = False, if_exists = 'append')

In [44]:
patrol_updates = patrol_deltas[patrol_deltas['dml_verb'] == 'U'][sql_cols]

In [45]:
patrol_updates.head()

,encounter_timestamp,encounter_datetime,site_id,location_adddesc,park_division,visit_reason,firstname_1,lastname_1,firstname_2,lastname_2,...,sd_patronscomplied,sd_patronsnocomply,sd_amenity,summonscount_a01,summonscount_a03,summonscount_a04,summonscount_a22,other_summonstype,other_summonscount,borough


In [46]:
sql_update(patrol_updates, 'tbl_dpr_patrol', engine, ['encounter_timestamp', 'site_id'])